In [1]:
from zeep import Client
import sqlite3
import json
import datetime

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('bank.db')

# Create a cursor object to interact with the database
cursor = conn.cursor()

In [2]:
def call_fetch_demand_service(filePath):
    try:
        # Define the service endpoint URL
        service_fetch_demand_url = "http://localhost:8000/fetchDemandService?wsdl"
        
        # Create a Zeep client for the service
        client = Client(service_fetch_demand_url)
        
        # Define the request parameters
        file_content = "BASE64_ENCODED_CONTENT"
        file_path = filePath
        
        # Create the SOAP request
        request = {
            'file_content': file_content,
            'file_path': file_path
        }

        client_id = -1
        house_props = ""
        
        # Call the service operation and store the response
        response_str = client.service.process_txt_file(**request)
        
        # You can process the response as needed
        print("Response : ", response_str)
        response = json.loads(response_str)
    
        # Extract data from the JSON request
        name = response["nom"]
        email = response["email"]
        telephone = response["numero"]
        address = response["adresse"]
        loan_amount = response["montant"]
        loan_duration = response["duree"]
        house_address = response["adresseMaison"]
        house_value = response["prixMaison"]
        house_desc = response["description"]

        current_datetime = datetime.datetime.now()
        current_date = current_datetime.date()
        date_demand = current_date.strftime("%Y-%m-%d")
    
        # Prepare queries
        add_demand = "INSERT INTO demands (client_id, loan, duration, house_id, date_demand, valid) VALUES (?, ?, ?, ?, ?, ?)"
    
        extract_client_id = "SELECT client_id FROM clients WHERE email = ?"

        extract_house_props = "SELECT * FROM houses WHERE address = ?";

        # Track down the client in the database
        # client_id = cursor.execute(extract_client_id, email)
        client_id_result = cursor.execute(extract_client_id, (email,)).fetchone()
        if client_id_result:
            client_id = client_id_result[0]
            print("Client ID : ", client_id)
        else:
            print("Client ID not found.")
            return

        # Track down the house concerned in the database
        house_props_result = cursor.execute(extract_house_props, (house_address,)).fetchone()
        if house_props_result:
            house_props = house_props_result
            print("House Properties : ", house_props)
        else:
            print("House not found.")
            return

    
        # Add to table demands
        cursor.execute(add_demand, (client_id, loan_amount, loan_duration, house_props[0], date_demand, False))
        
        # Commit the changes to the database
        conn.commit()
    
        return [response, client_id, house_props]
    except Exception as e:
        return "Failed to execute the first service" + str(e)

In [3]:
# Define a function to calculate the loan score
def calculate_loan_score(client_id):
    # Define the service endpoint URL
    service_url = "http://localhost:8000/calculScoreService?wsdl"

    # Create a Zeep client for the service
    client = Client(service_url)

    # Prepare queries
    retrieve_info = "SELECT debts, late_payments, recent_falls FROM properties WHERE client_id = ?"

    [dettes_en_cours, paiement_en_retard, antecedent_faillite] = cursor.execute(retrieve_info, (client_id,)).fetchone()  
    
    # Create the SOAP request with dynamically retrieved values
    request = {
        'dettes_en_cours': dettes_en_cours,
        'paiement_en_retard': paiement_en_retard,
        'antecedent_faillite': antecedent_faillite
    }

    # Call the service operation and store the response
    response = client.service.calculate_loan_score(**request)

    # Process the response as needed
    return response


In [4]:
# Define a function to analyse yearly income-outcome
def analyse_rev_dep(client_id):
    # Define the service endpoint URL
    service_url = "http://localhost:8000/analyseDemandeService?wsdl"

    # Create a Zeep client for the service
    client = Client(service_url)

    # Prepare queries
    retrieve_info = "SELECT yearly_income, yearly_outcome FROM properties WHERE client_id = ?"

    [yearly_income, yearly_outcome] = cursor.execute(retrieve_info, (client_id,)).fetchone()  
    
    # Create the SOAP request with dynamically retrieved values
    request = {
        'revenus': yearly_income,
        'depenses': yearly_outcome
    }

    # Call the service operation and store the response
    response = client.service.evaluer_capacite_remboursement(**request)

    # Process the response as needed
    return response


In [5]:
# Define a function to evaluate the house (eligible or not to be sold)
def verif_conform_house(house_id):
    # Define the service endpoint URL
    service_url = "http://localhost:8000/verifConformHouse?wsdl"

    # Create a Zeep client for the service
    client = Client(service_url)

    # Prepare queries
    retrieve_info = "SELECT land_dispute , complies_with_regulations, eligible_for_loan FROM houses WHERE house_id = ?"

    [land_dispute , complies_with_regulations, eligible_for_loan] = cursor.execute(retrieve_info, (house_id,)).fetchone()  
    
    # Create the SOAP request with dynamically retrieved values
    request = {
        'land_dispute': land_dispute,
        'complies_with_regulations': complies_with_regulations,
        'eligible_for_loan': eligible_for_loan
    }

    # Call the service operation and store the response
    response = client.service.verifConformHouse(**request)

    # Process the response as needed
    return response


In [6]:
# A TESTER !!

# Define a function to estimate the price of the house
def estim_val_prop(region, codePostal , surface , nbChambres):
    # Define the service endpoint URL
    service_url = "http://localhost:8000/valHouseEstim?wsdl"

    # Create a Zeep client for the service
    client = Client(service_url)

    # Prepare queries
    retrieve_info = """SELECT 
                        estimated_value FROM houses 
                        WHERE region = ? 
                        AND code_postal = ? 
                        AND num_of_chambers = ? 
                        AND area between ? and ?"""
    values = cursor.execute(retrieve_info, (region,codePostal,nbChambres,surface-100,surface+100,)).fetchall()  
    print(values)

    floats = []

    for item in values:
        floats.append(item[0])
    
    # Create the SOAP request with dynamically retrieved values
    request = {
        'input_list': floats
    }

    print("Floats : ", floats)
    
    # Call the service operation and store the response
    response = client.service.valHouseEstim(**request)

    print("Estimated response:", response) 

    # Process the response as needed
    return response


In [7]:
def controller(filePath):
    try:
        # Call the service that parses the demand
        [response, client_id, house_props] = call_fetch_demand_service(filePath)
        
        house_id = house_props[0]
        house_addr = house_props[10]
        house_est_val = house_props[2]
        house_area = house_props[3]
        house_nb_chambers = house_props[4]
        house_region = house_props[9]

        # Call the service that calculates the loan score
        calc_loan = calculate_loan_score(client_id)
        
        # Call the service that analyses the yearly income outcome rapport
        analyse_rev = analyse_rev_dep(client_id)

        if analyse_rev == "Le client a une capacité de remboursement positive.":
            isAllowed = True
        else:
            isAllowed = False
        
        # Call the service that estimates the price of the house
        est_prop = estim_val_prop(house_region, house_addr, house_area, house_nb_chambers)
        
        # Call the service that evaluates the elligibility of the house
        verify_conf = verif_conform_house(house_id)
        
        # Process the result
        print("Loan score : ", calc_loan)
        print("Analyse revenues and depenses : ", analyse_rev)
        print("House price estimate : ", est_prop)
        print("Le bien est-il conforme ?? : ", verify_conf)
        print("Is allowed : ", isAllowed)

        # Il faut que le prix proposé dans la demande du client soit plus que le prix estimé - 20000 (au moins : prix - 20000)
        if isAllowed and verify_conf and calc_loan >= 20 and house_est_val >=  (est_prop - 20000):
            result = True
        else:
            result = False
        
        # Close connection
        conn.close()
        return result
    except Exception as e:
        raise e


In [8]:
import tkinter as tk
from tkinter import filedialog as fd
from tkinter.messagebox import showinfo

def select_file():
    try:
        filetypes = (
            ('Text files', '*.txt'),
            ('All files', '*.*')
        )
    
        filename = fd.askopenfilename(
            title='Open a file',
            initialdir='./demands',
            filetypes=filetypes)
    
        return filename
    except Exception as e:
        print("Error :", e)

def return_selected_file():
    try:
        filePath = select_file()
        if filePath:
            showinfo(
                title='Selected File',
                message=filePath
            )
        return filePath
    except Exception as e:
        print("Error : ", e)

In [9]:
if __name__ == "__main__":
    try:
        filePath = return_selected_file()
        #filePath = './demands/client_1.txt'
        print("Demand chosen : ", filePath)
    
        # Call the controller to treat the demand
        result = controller(filePath)

        if result:
            print("Loan granted!")
        else:
            print("Loan denied!")
    except Exception as e:
        print("Error : ", e)

Demand chosen :  C:/Users/pault/Documents/WebServices/soap/demands/client_3.txt
Response :  {"nom": "John Doe", "email": "john@example.com", "adresse": "123 Main Street, Anytown, USA", "numero": "", "duree": "", "description": "", "montant": 0.0, "adresseMaison": "555 Birch Street", "prixMaison": 180000}
Client ID :  1
House Properties :  (6, '555 Birch Street', 270000.0, 2200.0, 3, 1, 0, 0, 0, 'South', '22222')
[(270000.0,)]
Floats :  [270000.0]
Estimated response: 270000.0
Loan score :  30
Analyse revenues and depenses :  Le client a une capacité de remboursement positive.
House price estimate :  270000.0
Le bien est-il conforme ?? :  False
Is allowed :  True
Loan denied!
